# Build Surrogate model for reaction propensity with casadi

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import h5py
import os
import sys

In [3]:
directory = '../result/'

ks, covs = [], []
for filename in os.listdir(directory):
    f = h5py.File(os.path.join(directory, filename), "r")
    
    print(filename, f['ks'].value, f['coverage'].value)
    
    ks.append(f['ks'].value)
    covs.append(f['coverage'].value)
    f.close()

covs = np.array(covs)
ks = np.array(ks)

/home/thj2009/bin/miniconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  import sys
/home/thj2009/bin/miniconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:9: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  if __name__ == '__main__':
/home/thj2009/bin/miniconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:10: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  # Remove the CWD from sys.path while we load stuff.


rate_0.1528.h5 [0.48003951 0.29128469 1.3968304 ] [0.8472 0.1528]
rate_0.2884.h5 [1.4874236  1.20040041 0.87952688] [0.7116 0.2884]
rate_0.3808.h5 [2.62018537 3.04499983 0.5309135 ] [0.6192 0.3808]
rate_0.8316.h5 [7.51666723e+01 1.26374657e+03 1.00118000e-02] [0.1684 0.8316]
rate_0.536.h5 [8.26605440e+00 9.83141472e+01 3.90801500e-02] [0.464 0.536]
rate_0.2504.h5 [1.12256072 0.82235198 1.02278007] [0.7496 0.2504]
rate_0.0832.h5 [0.20854533 0.11452124 1.66931373] [0.9168 0.0832]
rate_0.2648.h5 [1.25047765 0.94513119 0.96831692] [0.7352 0.2648]
rate_0.6552.h5 [4.57143956e+01 3.32831390e+02 4.72394000e-02] [0.3448 0.6552]
rate_0.2488.h5 [1.11058709 0.81143327 1.02888432] [0.7512 0.2488]
rate_0.6568.h5 [4.61724811e+01 3.37887735e+02 4.68765750e-02] [0.3432 0.6568]
rate_0.3236.h5 [1.90748974 1.72761885 0.74785332] [0.6764 0.3236]
rate_0.9788.h5 [1.67606235e+01 2.84969617e+03 1.21925000e-04] [0.0212 0.9788]
rate_0.2736.h5 [1.33767    1.03772141 0.93521017] [0.7264 0.2736]
rate_0.508.h5 [ 4.0

In [ ]:
from caslearn import NN
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
    
def trian_NN(X, y):
    modelname = 'nn_lograte_' + str(pair_entry[idx]) + '.json'
    
    # modify the strucutre of neural networks
    nn = NN(nin=2, nout=1, nhidden=2, nhDList=[3, 3])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

    nn.fit([X_train], [y_train])
    # nn.save_model(modelname)

    y_test_pred = nn.predict(X_test)
    y_train_pred = nn.predict(X_train)
    # plot err
    err = y_test.flatten() - y_test_pred.flatten()
    rmse = np.sqrt(np.mean(err ** 2))


    fig = plt.figure(dpi=200, figsize=(2.5, 2.5))
    ax = fig.gca()
    ax.set_aspect('equal')
    min_ = np.min(y_test.flatten().tolist() + y_test_pred.flatten().tolist() + y_train.flatten().tolist() + y_train_pred.flatten().tolist())
    max_ = np.max(y_test.flatten().tolist() + y_test_pred.flatten().tolist() + y_train.flatten().tolist() + y_train_pred.flatten().tolist())
    d = max_ - min_
    r = [min_ - 0.05 * d, max_ + 0.05 * d]

    ax.plot(y_train, y_train_pred, 'bs', ms=4)
    ax.plot(y_test, y_test_pred, 'ro', ms=4)

    ax.plot(r, r, 'k:')

    ax.text(0.1, 0.85, "RMSE = %.2e" % rmse, transform=ax.transAxes)

    ax.set_xlim(r)
    ax.set_ylim(r)
    ax.set_title(pair_entry[idx])
    ax.tick_params(which='both', direction='in')
    ax.set_xlabel('MC')
    ax.set_ylabel('NN')
    # fit on overall dataset and save the model
    nn.fit([X], [y])
    nn.save_model(modelname)

In [ ]:
pair_entry = [(1, 0), (1), (0, 0)]

In [5]:

for idx in range(3):

    # load the dataset
    X, y = [], []
    for i in range(len(ks)):
        try:
            cov1 = covs[i, pair_entry[idx][0]]
            cov2 = covs[i, pair_entry[idx][1]]
        except:
            cov1 = covs[i, pair_entry[idx]]
            cov2 = 1
        X.append([covs[i, 0], covs[i, 1]])
        if ks[i, idx] == 0:
            print("**************")
            y.append(0)
        else:
            y.append(np.log(ks[i, idx] / (cov1 * cov2)))

    X = np.array(X)
    y = np.array(y).reshape(-1, 1)


    trian_NN(X, y)


